In [1]:
# ✅ STEP 1: Install Required Packages
!pip install -q transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.8 MB/s eta 0:00:00


In [2]:
# ✅ STEP 2: Import Required Libraries
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [3]:
# ✅ STEP 3: Load Your Datasets
recipes_df = pd.read_csv("/content/70000_recipes_nutrients.csv")
health_df = pd.read_csv("/content/health_age_data_70000_synthetic.csv")

In [4]:
# ✅ STEP 4: Clean Ingredients
def clean_ingredients(raw_ingredients):
    if pd.isna(raw_ingredients): return ""
    items = [re.sub(r"[^a-zA-Z ]", "", x).strip().lower() for x in str(raw_ingredients).split(',')]
    items = [i for i in items if len(i) > 1 and not i.isdigit()]
    return ', '.join(sorted(set(items)))

recipes_df["cleaned_ingredients"] = recipes_df["ingredients"].apply(clean_ingredients)


In [5]:
# ✅ STEP 5: Load Bloom Model (560M) and Tokenizer
model_name = "bigscience/bloom-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [8]:
# ✅ STEP 6: Define Prompt Template
def create_prompt(dish_name, ingredients, user_profile):
    age = user_profile['Ages'] # Corrected column name
    condition = user_profile['Disease'] # Using 'Disease' as condition

    # Example: Include rules based on condition
    avoid_map = {
        "Diabetes": "sugar, white rice, honey",
        "Heart Disease": "red meat, salt, fried foods"
    }
    include_map = {
        "Diabetes": "spinach, cinnamon, olive oil",
        "Heart Disease": "oats, berries, nuts"
    }

    # Extract the first condition mentioned in the 'Disease' column
    condition_list = [c.strip() for c in condition.split(',')]
    primary_condition = condition_list[0] if condition_list else "None"


    avoid = avoid_map.get(primary_condition, "sugar")
    include = include_map.get(primary_condition, "spinach")


    return (
        f"Suggest a healthier version of the dish '{dish_name}' for a {age}-year-old with {primary_condition}. "
        f"It currently contains: {ingredients}. "
        f"Avoid: {avoid}. Try to include: {include}. "
        f"Explain the ingredient changes briefly."
    )

# ✅ STEP 7: Generate Output using Bloom
def generate_recipe(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = model.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ STEP 8: Test With One Example
sample_recipe = recipes_df.iloc[0]
sample_user = health_df.iloc[0]

prompt = create_prompt(
    dish_name=sample_recipe["recipe_name"],
    ingredients=sample_recipe["cleaned_ingredients"],
    user_profile=sample_user
)

print("🔹 Prompt:\n", prompt)
print("\n🌟 Bloom LLM Output:\n", generate_recipe(prompt))

🔹 Prompt:
 Suggest a healthier version of the dish 'stalker pasta' for a 52-year-old with Diabetes. It currently contains: c chopped fresh parsley leaves, c freshly grated parmesan cheese, c red wine, diced, lb ground pork, oz mushrooms white or baby portobellos, oz pancetta or regular bacon, such as pinot noir or cabernet, tbsp olive oil, trimmed and thinly sliced, washed. Avoid: sugar, white rice, honey. Try to include: spinach, cinnamon, olive oil. Explain the ingredient changes briefly.

🌟 Bloom LLM Output:
 Suggest a healthier version of the dish 'stalker pasta' for a 52-year-old with Diabetes. It currently contains: c chopped fresh parsley leaves, c freshly grated parmesan cheese, c red wine, diced, lb ground pork, oz mushrooms white or baby portobellos, oz pancetta or regular bacon, such as pinot noir or cabernet, tbsp olive oil, trimmed and thinly sliced, washed. Avoid: sugar, white rice, honey. Try to include: spinach, cinnamon, olive oil. Explain the ingredient changes briefl

In [10]:
# ✅ STEP 8: Example – Dish: Butter Chicken | Age: 35 | Condition: Heart Disease
custom_dish_name = "butter chicken"
custom_ingredients = "butter, cream, chicken thighs, tomato puree, garam masala, salt"
custom_age = 35
custom_condition = "heart disease"

custom_user_profile = {'Ages': custom_age, 'Disease': custom_condition} # Create a dictionary for the user profile

custom_prompt = create_prompt( # Corrected function name
    dish_name=custom_dish_name,
    ingredients=custom_ingredients,
    user_profile=custom_user_profile # Pass the user profile dictionary
)

print("🔹 Prompt:\n", custom_prompt)
print("\n🌟 Bloom LLM Output:\n", generate_recipe(custom_prompt)) # Corrected function name

🔹 Prompt:
 Suggest a healthier version of the dish 'butter chicken' for a 35-year-old with heart disease. It currently contains: butter, cream, chicken thighs, tomato puree, garam masala, salt. Avoid: sugar. Try to include: spinach. Explain the ingredient changes briefly.

🌟 Bloom LLM Output:
 Suggest a healthier version of the dish 'butter chicken' for a 35-year-old with heart disease. It currently contains: butter, cream, chicken thighs, tomato puree, garam masala, salt. Avoid: sugar. Try to include: spinach. Explain the ingredient changes briefly. For the butter chicken, use a medium and high fat mixture. For the cream, use a medium and low fat mixture. For the chicken thighs, use a medium and low fat mixture. For the tomato puree, use a medium and low fat mixture. For the garam masala, use a medium and low fat mixture. For the salt, use a medium and low fat mixture. The chicken, butter and cream are used in a regular way. For the salt, use 1/4 cup
